Version: 0.0.2  Updated date: 07/05/2024
Conda Environment : py-snowpark_df_ml_fs-1.15.0_v1

# Getting Started with Snowflake Feature Store
We will use the Use-Case to show how Snowflake Feature Store (and Model Registry) can be used to maintain & store features, retrieve them for training and perform micro-batch inference.

In the development (TRAINING) enviroment we will 
- create FeatureViews in the Feature Store that maintain the required customer-behaviour features.
- use these Features to train a model, and save the model in the Snowflake model-registry.
- plot the clusters for the trained model to visually verify. 

In the production (SERVING) environment we will
- re-create the FeatureViews on production data
- generate an Inference FeatureView that uses the saved model to perform incremental inference

# Feature Engineering & Model Training

In [1]:
%load_ext autoreload
%autoreload 2

#### Notebook Packages

In [2]:
# Python packages
import os
import json
import timeit

# SNOWFLAKE
# Snowpark
from snowflake.snowpark import Session, DataFrame, Window, WindowSpec

import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T

# Snowflake Feature Store
from snowflake.ml.feature_store import (
    FeatureView,
    Entity)

# COMMON FUNCTIONS
from useful_fns import check_and_update, get_latest, create_ModelRegistry, create_FeatureStore, create_SF_Session, get_spine_df
from useful_fns import run_sql

/opt/miniconda3/envs/py-snowpark_df_ml_fs/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Setup Snowflake connection and database parameters

In [3]:
# Schemas
tpcxai_training_schema     = 'SERVING'

In [4]:
fs_qs_role, tpcxai_database, tpcxai_training_schema, session, warehouse_env = create_SF_Session(tpcxai_training_schema)

You might have more than one threads sharing the Session object trying to update sql_simplifier_enabled. Updating this while other tasks are running can potentially cause unexpected behavior. Please update the session configuration before starting the threads.



Connection Established with the following parameters:
User                        : JARCHEN
Role                        : "FS_QS_ROLE"
Database                    : "TPCXAI_SF0001_QUICKSTART_INC"
Schema                      : "SERVING"
Warehouse                   : "TPCXAI_SF0001_QUICKSTART_WH"
Snowflake version           : 9.26.0
Snowpark for Python version : 1.38.0 



## MODEL DEVELOPMENT
* Create Snowflake Model-Registry
* Create Snowflake Feature-Store
* Establish and Create CUSTOMER Entity in the development Snowflake FeatureStore
* Create Source Data references and perform basic data-cleansing
* Create & Run Preprocessing Function to create features
* Create FeatureView_Preprocess from Preprocess Dataframe SQL
* Create training data from FeatureView_Preprocess (asof join)
* Create & Fit Snowpark-ml pipeline 
* Save model in Model Registry
* 'Verify' and approve model
* Create new FeatureView_Model_Inference with Transforms UDF + KMeans model

In [5]:
# Set the Schema
tpcxai_schema = tpcxai_training_schema

# Create/Reference Snowflake Model Registry - Common across Environments
mr = create_ModelRegistry(session, tpcxai_database, 'MODEL_1')

# Create/Reference Snowflake Feature Store for Training (Development) Environment
fs = create_FeatureStore(session, tpcxai_database, f'''_{tpcxai_schema}_FEATURE_STORE''', warehouse_env)


Model Registry (MODEL_1) already exists
Feature Store (_SERVING_FEATURE_STORE) already exists


 ### Create & Load Source Data

### Create Preprocessing FeatureView from Preprocess Dataframe (SQL)

In [6]:
ppd_fv_name    = "FV_UC01_PREPROCESS"
ppd_fv_version = "V_1"

fv_uc01_preprocess = fs.get_feature_view(name=ppd_fv_name,version=ppd_fv_version)
spine = fv_uc01_preprocess

In [7]:
# You can also use the following to retrieve a Feature View instance for use within Python
FV_UC01_PREPROCESS_V_1 = fs.get_feature_view(ppd_fv_name, 'V_1')
# We can look at the FeatureView's contents with
FV_UC01_PREPROCESS_V_1.feature_df.show(10)

---------------------------------------------------------------------------------------------
|"O_CUSTOMER_SK"  |"LATEST_ORDER_DATE"  |"FREQUENCY"  |"RETURN_RATIO"  |"RETURN_ROW_PRICE"  |
---------------------------------------------------------------------------------------------
|1293             |2025-05-01           |1.0          |0.899           |49.49               |
|5139             |2025-05-16           |3.0          |0.879           |26.333              |
|6998             |2025-07-09           |2.0          |0.701           |61.263              |
|4981             |2025-06-13           |2.0          |0.772           |66.82               |
|6691             |2025-01-03           |1.0          |0.777           |9.635               |
|4342             |2025-01-24           |1.0          |0.644           |24.23               |
|6417             |2025-08-07           |1.5          |0.821           |68.513              |
|5287             |2025-04-01           |1.0          |0.66 

### Create training data Dataset from FeatureView_Preprocess

In [8]:
# Create Spine
spine_sdf = get_spine_df(spine)
spine_sdf.sort('O_CUSTOMER_SK').show(5)

-------------------------------------------
|"O_CUSTOMER_SK"  |"ASOF_DATE"  |"COL_1"  |
-------------------------------------------
|0                |2025-06-19   |values1  |
|1                |2025-08-15   |values1  |
|2                |2025-07-18   |values1  |
|3                |2025-08-15   |values1  |
|4                |2024-09-28   |values1  |
-------------------------------------------



In [9]:
# Generate_Dataset
training_dataset = fs.generate_dataset( name = 'UC01_TRAINING',
                                        spine_df = spine_sdf, 
                                        features = [fv_uc01_preprocess], 
                                        spine_timestamp_col = 'ASOF_DATE'
                                        )                                     
# Create a snowpark dataframe reference from the Dataset
training_dataset_sdf = training_dataset.read.to_snowpark_dataframe()
# Display some sample data
training_dataset_sdf.sort('O_CUSTOMER_SK').show(5)


---------------------------------------------------------------------------------------------------
|"O_CUSTOMER_SK"  |"ASOF_DATE"  |"COL_1"  |"FREQUENCY"  |"RETURN_RATIO"      |"RETURN_ROW_PRICE"  |
---------------------------------------------------------------------------------------------------
|0                |2025-06-19   |values1  |2.5          |0.7400000095367432  |70.84600067138672   |
|1                |2025-08-15   |values1  |4.0          |0.7730000019073486  |109.947998046875    |
|2                |2025-07-18   |values1  |2.5          |0.7580000162124634  |35.47999954223633   |
|3                |2025-08-15   |values1  |2.0          |0.75                |30.969999313354492  |
|4                |2024-09-28   |values1  |2.0          |0.6069999933242798  |11.5649995803833    |
---------------------------------------------------------------------------------------------------



### Fit Snowpark-ML Transforms & Model using Fileset training data

We need to fit the transformer over the training Fileset to ensure we are using the same input global values for transforming and training, and later inference with the model.

The transforms here are model-specific and persisted within the model-pipeline, and not stored in the Feature Store.

In [10]:
weights = [0.6, 0.4]
training_dataset_sdf = training_dataset_sdf.with_column("FREQUENCY", F.round(F.col("FREQUENCY"), 3))
training_dataset_sdf = training_dataset_sdf.with_column("RETURN_RATIO", F.round(F.col("RETURN_RATIO"), 3))
training_dataset_sdf = training_dataset_sdf.with_column("RETURN_ROW_PRICE", F.round(F.col("RETURN_ROW_PRICE"), 3))
training_dataset_sdf = training_dataset_sdf.select(['RETURN_RATIO', 'FREQUENCY', 'RETURN_ROW_PRICE'])

train_df, test_df = training_dataset_sdf.random_split(weights, seed=42) # Using a seed for reproducibility


In [11]:
# Get and set default for latest version of the model
model_name = "MODEL_1.UC01_SNOWFLAKEML_XGB_REGRESSOR_MODEL"
m = mr.get_model(model_name)
latest_version = m.show_versions().iloc[-1]['name']
mv = m.version(latest_version)
m.default = latest_version

In [12]:
explanations = mv.run(train_df, function_name="explain")
explanations = explanations.with_column_renamed(F.col('"""RETURN_RATIO_explanation"""'), "RETURN_RATIO_EXPLANATION")
explanations = explanations.with_column_renamed(F.col('"""FREQUENCY_explanation"""'), "FREQUENCY_EXPLANATION")
explanations = explanations.limit(2000)

In [13]:
explanations.show(2)

------------------------------------------------------------------------------------------------------------
|"RETURN_RATIO"  |"FREQUENCY"  |"RETURN_ROW_PRICE"  |"RETURN_RATIO_EXPLANATION"  |"FREQUENCY_EXPLANATION"  |
------------------------------------------------------------------------------------------------------------
|0.723           |1.5          |35.323              |15.956960678100586          |7.700674533843994        |
|0.879           |3.0          |39.02               |2.4679486751556396          |-0.2201704978942871      |
------------------------------------------------------------------------------------------------------------



In [14]:
from snowflake.ml.monitoring.explain_visualize import plot_violin
plot_violin(
    explanations.select(['RETURN_RATIO_EXPLANATION', 'FREQUENCY_EXPLANATION']),
    explanations.select(["RETURN_RATIO", "FREQUENCY"]),
    (600,150)
)

alt.Chart(...)

# Observability 

In [15]:
## Create & Register Inference-FeatureView to run scheduled Inference
inf_fvname = "FV_UC01_INFERENCE_RESULT"
inf_fv_version = "V_1"

fv_uc01_inference_result = fs.get_feature_view(name= inf_fvname, version= inf_fv_version)
inference_input_sdf = fs.read_feature_view(fv_uc01_inference_result)

In [16]:
inference_input_sdf.show()

-------------------------------------------------------------------------------------------------------------------------
|"O_CUSTOMER_SK"  |"LATEST_ORDER_DATE"  |"RETURN_ROW_PRICE"  |"RETURN_RATIO"  |"FREQUENCY"  |"OUTPUT_RETURN_ROW_PRICE"  |
-------------------------------------------------------------------------------------------------------------------------
|5809             |2025-03-17           |25.913              |0.704           |2.0          |51.049076080322266         |
|2195             |2025-02-26           |70.87               |0.927           |1.5          |54.010658264160156         |
|5280             |2025-04-30           |11.445              |0.811           |3.0          |51.58135986328125          |
|2829             |2025-07-09           |44.635              |0.91            |2.0          |43.89048767089844          |
|6564             |2025-05-20           |64.268              |0.725           |2.5          |53.434486389160156         |
|2263             |2025-

In [ ]:
## Create & Register Inference-FeatureView to run scheduled Inference
monitoring_fs = create_FeatureStore(session, tpcxai_database, f'''MODEL_1''', warehouse_env)
### ORDER Entity
if "ORDER" not in json.loads(monitoring_fs.list_entities().select(F.to_json(F.array_agg("NAME", True))).collect()[0][0]):
    customer_entity = Entity(name="ORDER", join_keys=["O_CUSTOMER_SK"],desc="Primary Key for CUSTOMER ORDER")
    monitoring_fs.register_entity(customer_entity)
else:
    customer_entity = monitoring_fs.get_entity("ORDER")

monitoring_fvname = "FV_UC01_MONITORING"
monitoring_fv_version = "V_1"
try:
   fv_uc01_monitoring_result = monitoring_fs.get_feature_view(name= monitoring_fvname, version= monitoring_fv_version)
except:
   fv_uc01_inference_result = FeatureView(
        name= monitoring_fvname, 
        entities=[customer_entity], 
        feature_df=inference_input_sdf,
        refresh_freq="60 minute",
        refresh_mode="INCREMENTAL",
        desc="Inference Result for monitoring")
   
   fv_uc01_inference_result = monitoring_fs.register_feature_view(
         feature_view=fv_uc01_inference_result, 
         version= monitoring_fv_version, 
         block=True
   )

Feature Store (MODEL_1) created


In [22]:
# Check for the latest version of this model in registry, and increment version
mr_df = mr.show_models()
model_version = get_latest(mr_df, model_name)
mv = m.version(model_version)


In [23]:
# run_sql(f'''CREATE OR REPLACE TABLE {tpcxai_database}.{tpcxai_training_schema}.RETURN_PRICE_PRED
#     (ASOF_DATE DATE,
#      OR_PRODUCT_ID INTEGER,
#      OR_RETURN_QUANTITY INTEGER)
#      CLUSTER BY (OR_PRODUCT_ID, OR_ORDER_ID);
#     ''', session)

In [27]:
from snowflake.ml.monitoring.entities.model_monitor_config import ModelMonitorConfig, ModelMonitorSourceConfig

source_config = ModelMonitorSourceConfig(
    source="TPCXAI_SF0001_QUICKSTART_INC.MODEL_1.FV_UC01_MONITORING$V_1",
    timestamp_column="LATEST_ORDER_DATE",
    prediction_class_columns=["OUTPUT_RETURN_ROW_PRICE"],
    actual_class_columns=["RETURN_ROW_PRICE"],
    id_columns=["O_CUSTOMER_SK"],
)

# Set up config for ModelMonitor.
model_monitor_config = ModelMonitorConfig(
    model_version=mv,
    model_function_name="predict",
    background_compute_warehouse_name="COMPUTE_WH",
)

# Add a new ModelMonitor
model_monitor = mr.add_monitor(
    name=model_name, 
    source_config=source_config,
    model_monitor_config=model_monitor_config,
)
model_monitor


SnowparkSQLException: (1304): 01beeec0-3204-c944-0002-48ee00dc3aae: 002003 (02000): SQL compilation error:
Module 'null' does not exist or not authorized.

In [35]:
query = f"""
USE SCHEMA MODEL_1
"""
run_sql(query, session)


query = f"""
CREATE OR REPLACE MODEL MONITOR CHURN_MODEL_MONITOR
WITH
    MODEL=UC01_SNOWFLAKEML_XGB_REGRESSOR_MODEL
    VERSION=V_3
    FUNCTION=predict
    SOURCE=FV_UC01_MONITORING$V_1
    TIMESTAMP_COLUMN=LATEST_ORDER_DATE
    PREDICTION_SCORE_COLUMNS=(OUTPUT_RETURN_ROW_PRICE)  
    ACTUAL_SCORE_COLUMNS=(RETURN_ROW_PRICE)
    ID_COLUMNS=(O_CUSTOMER_SK)
    WAREHOUSE=COMPUTE_WH
    REFRESH_INTERVAL='1 min'
    AGGREGATION_WINDOW='1 day';
"""
run_sql(query, session)


USE SCHEMA MODEL_1
 
 [Row(status='Statement executed successfully.')] 



SnowparkSQLException: (1304): 01beeec7-3204-c944-0002-48ee00dc3b96: 091941 (0A000): SQL compilation error: line 0 at position -1:
Change tracking is not supported on dynamic tables with 'FULL' REFRESH_MODE unless the Dynamic Table has IMMUTABLE WHERE specified.

## CLEAN UP

In [ ]:
# session.close()

In [ ]:
from datetime import datetime
from zoneinfo import ZoneInfo
formatted_time = datetime.now(ZoneInfo("Australia/Melbourne")).strftime("%A, %B %d, %Y %I:%M:%S %p %Z")

print(f"The last run time in Melbourne is: {formatted_time}")